# Basic Video Scene Graph Generation

This notebook demonstrates how to generate scene graphs from video data using the m3sgg library.
We'll walk through the complete pipeline from video input to scene graph visualization.

## Overview

1. **Video Loading**: Load video data and extract frames
2. **Object Detection**: Detect objects in each frame
3. **Scene Graph Generation**: Generate relationships between objects
4. **Visualization**: Display the results

## Prerequisites

Make sure you have the required dependencies installed:
```bash
pip install torch torchvision opencv-python matplotlib networkx
```


In [ ]:
# Import required libraries
import sys
import os
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path

# Add the src directory to the path
sys.path.append(str(Path.cwd().parent / "src"))

# Import m3sgg components
from m3sgg.core.models.sttran import STTran
from m3sgg.core.datasets.action_genome import AG, cuda_collate_fn
from m3sgg.core.config import Config
from m3sgg.core.object_detector import detector
from m3sgg.core.evaluation_recall import BasicSceneGraphEvaluator

print("Libraries imported successfully!")


## 1. Setup and Configuration

First, let's set up the configuration and check if we have the required data and models.


In [ ]:
# Configuration
config = Config()
config.data_path = "../data/action_genome"  # Adjust path as needed
config.mode = "sgdet"  # Scene graph detection mode
config.datasize = 100  # Use smaller dataset for demo

# Check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Check if data path exists
if not os.path.exists(config.data_path):
    print(f"Warning: Data path {config.data_path} does not exist.")
    print("Please ensure the Action Genome dataset is available.")
else:
    print(f"Data path found: {config.data_path}")


## 2. Load Dataset

Load the Action Genome dataset for video scene graph generation.


In [ ]:
# Load dataset
try:
    dataset = AG(
        mode="test",
        datasize=config.datasize,
        data_path=config.data_path,
        filter_nonperson_box_frame=True,
        filter_small_box=False if config.mode == "predcls" else True,
    )
    
    dataloader = torch.utils.data.DataLoader(
        dataset, shuffle=False, num_workers=0, collate_fn=cuda_collate_fn
    )
    
    print(f"Dataset loaded successfully!")
    print(f"Number of samples: {len(dataset)}")
    print(f"Number of batches: {len(dataloader)}")
    
    # Get object classes
    obj_classes = dataset.obj_classes
    print(f"Number of object classes: {len(obj_classes)}")
    print(f"First 10 object classes: {obj_classes[:10]}")
    
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("This might be due to missing data files.")
    print("Please ensure the Action Genome dataset is properly set up.")
